In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/Users/olivierbougeant/prodigyfinance/projects/21_conversion_rate_synthetic_control/synthetic_control


/Users/olivierbougeant/prodigyfinance/venv/synthetic_control/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from datetime import datetime
import pandas as pd
from synthetic_control import SyntheticControl

# Read California proposition 99 dataset

In [471]:
df = pd.read_csv("./examples/california_prop99.csv")

In [472]:
df["Year"] = pd.to_datetime(df["Year"], format='%Y')

In [473]:
df = df.pivot(index="Year", columns="State",values="PacksPerCapita")

In [474]:
df.head()

State,Alabama,Arkansas,California,Colorado,Connecticut,Delaware,Georgia,Idaho,Illinois,Indiana,...,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,
1970-01-01,89.800003,100.300003,123.000000,124.800003,120.000000,155.000000,109.900002,102.400002,124.800003,134.600006,...,103.599998,92.699997,99.800003,106.400002,65.500000,122.599998,124.300003,114.500000,106.400002,132.199997
1971-01-01,95.400002,104.099998,121.000000,125.500000,117.599998,161.100006,115.699997,108.500000,125.599998,139.300003,...,115.000000,96.699997,106.300003,108.900002,67.699997,124.400002,128.399994,111.500000,105.400002,131.699997
1972-01-01,101.099998,103.900002,123.500000,134.300003,110.800003,156.300003,117.000000,126.099998,126.599998,149.199997,...,118.699997,103.000000,111.500000,108.599998,71.300003,138.000000,137.000000,117.500000,108.800003,140.000000
1973-01-01,102.900002,108.000000,124.400002,137.899994,109.300003,154.699997,119.800003,121.800003,124.400002,156.000000,...,125.500000,103.500000,109.699997,110.400002,72.699997,146.800003,143.100006,116.599998,109.500000,141.199997
1974-01-01,108.199997,109.699997,126.699997,132.800003,112.400002,151.300003,123.699997,125.599998,131.899994,159.600006,...,129.699997,108.400002,114.800003,114.699997,75.599998,151.800003,149.600006,119.900002,111.800003,145.800003


In [475]:
X = df.drop(columns="California")
y = df["California"]

# Generate predictions for the Synthetic Control group

In [476]:
sc = SyntheticControl(treatment_start=datetime(year=1989, month=1, day=1), fit_intercept=True, alpha=100, ci_fraction=0.5)

In [477]:
y_synth = sc.fit_predict(X, y)

In [478]:
y_ci = sc.get_confidence_interval(X, y)

In [479]:
y_ci.head()

,5,10,20,30,40,50,60,70,80,90,95
Year,,,,,,,,,,,
1970-01-01,113.024268,114.607882,116.895337,117.620794,118.092861,118.413063,118.840484,119.154834,119.664386,120.437903,120.517503
1971-01-01,116.184597,118.590287,119.729540,120.762757,121.157341,121.315101,121.608728,121.882626,122.065706,122.334233,122.444045
1972-01-01,121.725716,123.008954,123.763263,124.106355,124.399703,124.777967,125.035980,125.615256,126.036410,126.963788,127.631711
1973-01-01,123.015383,123.309728,123.602820,123.839372,124.301989,124.675228,124.963723,125.134203,125.408985,125.747989,126.370481
1974-01-01,123.484382,123.698644,124.334346,125.072051,125.631332,125.997012,126.444331,126.686438,127.021408,127.342119,127.689062


# Display predictions for the Synthetic Control group

In [501]:
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS

In [502]:
from synthetic_control.analysis import display_synthetic_control

In [ ]:
sc.

In [505]:
display_synthetic_control(y, y_ci, sc.treatment_start, treatment_name="California", y_axis="Packs of cigarettes per Capita")

# Testing for false positives

### Using a different treatment date

In [494]:
sc = SyntheticControl(treatment_start=datetime(year=1995, month=1, day=1), fit_intercept=True, alpha=1.0, ci_fraction=0.5)

In [495]:
y_ci = sc.get_confidence_interval(X, y)

In [496]:
display_synthetic_control(y, y_ci, sc.treatment_start, treatment_name="California", y_axis="Packs of cigarettes per Capita")

### Application to the state of Pennsylvania 

In [497]:
X = df.drop(columns="Pennsylvania")
y = df["Pennsylvania"]

In [498]:
sc = SyntheticControl(treatment_start=datetime(year=1989, month=1, day=1), fit_intercept=True, alpha=1.0, ci_fraction=0.5)

In [499]:
y_ci = sc.get_confidence_interval(X, y)

In [500]:
display_synthetic_control(y, y_ci, sc.treatment_start, treatment_name="Texas", y_axis="Packs of cigarettes per Capita")